In [31]:
import numpy as np
import pandas as pd

import seaborn as sns

from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import PatternFill
from openpyxl.drawing.image import Image


In [32]:

def find_redundant_pairs(correlation_matrix, threshold):
    redundant_pairs = []
    # Iterate through the upper triangle of the correlation matrix
    for i in range(len(correlation_matrix.columns)):
        for j in range(i+1, len(correlation_matrix.columns)):
            if abs(correlation_matrix.iloc[i, j]) >= threshold:
                redundant_pairs.append((correlation_matrix.columns[i], correlation_matrix.columns[j]))
    return redundant_pairs

In [33]:
# Read correlation matrix from CSV file
correlation_matrix_df = pd.read_excel('filt_CorrMatrix-Daily-AllFractures-wrtOHS&OKS_heatmap.xlsx', index_col=0)
correlation_matrix_df.head()

,position-count,position-duration,position-maximum-distance,motion-sum,motion-ratio,motion-mean,motion-max,motion-max-timestamp,heartrate-min,heartrate-max,...,oks-07,oks-08,oks-09,oks-10,oks-11,oks-12,oks,tug,chairstand,sex
position-count,1.000000,0.972221,0.807925,0.140849,0.000000,0.159361,0.156295,0.000000,0.00000,0.000000,...,0.000000,0.218864,0.148139,0.214737,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
position-duration,0.972221,1.000000,0.769776,0.132288,0.000000,0.149009,0.146339,0.000000,0.00000,0.000000,...,0.000000,0.204809,0.150876,0.230606,0.000000,0.121985,0.000000,0.000000,0.000000,0.000000
position-maximum-distance,0.807925,0.769776,1.000000,0.210993,0.000000,0.223331,0.210109,0.000000,0.00000,0.000000,...,-0.174589,0.204322,0.000000,0.243063,0.000000,0.000000,0.000000,0.000000,0.000000,-0.143429
motion-sum,0.140849,0.132288,0.210993,1.000000,0.339678,0.920581,0.923681,0.436195,0.00000,0.122864,...,-0.295007,0.402982,0.299288,0.631509,0.210568,0.352965,0.280779,0.402421,-0.226646,0.000000
motion-ratio,0.000000,0.000000,0.000000,0.339678,1.000000,0.000000,0.000000,0.000000,-0.26763,-0.145652,...,-0.489956,0.000000,-0.376669,0.171569,-0.502535,-0.278429,-0.442033,0.162845,-0.351745,-0.206125


In [34]:
# Find redundant pairs
redundant_pairs = find_redundant_pairs(correlation_matrix_df, threshold=0.9)

In [35]:
redundant_pairs_df = pd.DataFrame(redundant_pairs, columns=['Column 1', 'Column 2'])
redundant_pairs_df

,Column 1,Column 2
0,position-count,position-duration
1,position-count,Time spent outside (mins)
2,position-count,distance travelled from home
3,position-duration,Time spent outside (mins)
4,position-duration,distance travelled from home
5,motion-sum,motion-mean
6,motion-sum,motion-max
7,motion-mean,motion-max
8,step-sum,step-max
9,step-mean,step-max


In [36]:
columns_to_drop = redundant_pairs_df['Column 2'].explode().unique()

In [37]:
columns_to_drop

array(['position-duration', 'Time spent outside (mins)',
       'distance travelled from home', 'motion-mean', 'motion-max',
       'step-max', 'l2_sum', 'l2_energy', 'l2_skewness', 'l2_rms',
       'l2_n_above_mean', 'l2_n_below_mean', 'l2_entropy',
       'l2_perm_entropy', 'l2_iqr', 'l2_ptp', 'l2_lineintegral',
       'l2_pct_95'], dtype=object)

In [38]:
correlation_matrix_df = correlation_matrix_df.drop(columns=columns_to_drop, index=columns_to_drop)
correlation_matrix_df.to_excel('CorrMatrix-Daily-AllFractures-wrtOHS&OKS-thres-0.9.xlsx')

In [39]:
correlation_matrix_df

,position-count,position-maximum-distance,motion-sum,motion-ratio,motion-max-timestamp,heartrate-min,heartrate-max,heartrate-mean,heartrate-std,totalsleepduration,...,oks-07,oks-08,oks-09,oks-10,oks-11,oks-12,oks,tug,chairstand,sex
position-count,1.000000,0.807925,0.140849,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.218864,0.148139,0.214737,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
position-maximum-distance,0.807925,1.000000,0.210993,0.000000,0.000000,0.000000,0.000000,0.133428,0.000000,0.000000,...,-0.174589,0.204322,0.000000,0.243063,0.000000,0.000000,0.000000,0.000000,0.000000,-0.143429
motion-sum,0.140849,0.210993,1.000000,0.339678,0.436195,0.000000,0.122864,0.000000,0.000000,-0.232538,...,-0.295007,0.402982,0.299288,0.631509,0.210568,0.352965,0.280779,0.402421,-0.226646,0.000000
motion-ratio,0.000000,0.000000,0.339678,1.000000,0.000000,-0.267630,-0.145652,-0.274434,0.000000,0.000000,...,-0.489956,0.000000,-0.376669,0.171569,-0.502535,-0.278429,-0.442033,0.162845,-0.351745,-0.206125
motion-max-timestamp,0.000000,0.000000,0.436195,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,-0.223082,...,0.000000,0.229028,0.253952,0.179267,0.266482,0.270039,0.341299,0.222999,0.000000,0.264915
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
oks-12,0.000000,0.000000,0.352965,-0.278429,0.270039,0.387567,0.370899,0.442963,0.000000,-0.153119,...,0.424400,0.319516,0.547086,0.395057,0.652800,1.000000,0.704893,-0.126480,0.239828,0.422252
oks,0.000000,0.000000,0.280779,-0.442033,0.341299,0.319038,0.239699,0.329499,0.000000,0.000000,...,0.380855,0.541133,0.821926,0.254830,0.885465,0.704893,1.000000,0.000000,0.216994,0.421711
tug,0.000000,0.000000,0.402421,0.162845,0.222999,0.000000,0.000000,0.000000,0.121149,-0.593562,...,-0.132440,-0.206147,-0.189662,0.000000,0.000000,-0.126480,0.000000,1.000000,-0.532319,-0.596897
chairstand,0.000000,0.000000,-0.226646,-0.351745,0.000000,0.169451,0.000000,0.187696,0.000000,0.307176,...,0.383099,0.000000,0.274197,0.000000,0.278183,0.239828,0.216994,-0.532319,1.000000,0.389841


In [40]:
# Save the correlation matrix to an Excel file with heatmap
excel_filename = 'CorrMatrix-Daily-AllFractures-wrtOHS&OKS-thres-0.9-heatmap.xlsx'
correlation_matrix_df.to_excel(excel_filename, sheet_name='Correlation Matrix')

# Open the Excel workbook and select the active sheet
wb = Workbook()
ws = wb.active

# Create a color map for seaborn heatmap
cmap = sns.color_palette("coolwarm", as_cmap=True)

# Apply color formatting to the Excel cells based on the seaborn heatmap
for row_idx, row in enumerate(correlation_matrix_df.index, start=2):
    for col_idx, col in enumerate(correlation_matrix_df.columns, start=2):
        cell = ws.cell(row=row_idx, column=col_idx)
        value = correlation_matrix_df.at[row, col]
        # Normalize the correlation value to the range [0, 1]
        normalized_value = (value + 1) / 2
        # Map the normalized value to RGB color using the seaborn colormap
        rgb_color = cmap(normalized_value)[:3]
        # Convert RGB to aRGB by adding an alpha channel (255)
        argb_color = (255,) + tuple(int(255 * x) for x in rgb_color)
        # Convert aRGB to HEX
        hex_color = '%02x%02x%02x%02x' % argb_color
        # Apply fill color to the cell
        cell.fill = PatternFill(start_color=hex_color, end_color=hex_color, fill_type="solid")

# Save the modified Excel file
wb.save(excel_filename)

wb.close()